Collect from: https://www.cboe.com/us/options/market_statistics/daily/

In [1]:
import pandas as pd
import datetime

url = "https://www.cboe.com/us/options/market_statistics/daily/"
filename_ratios = "./data/putcall_ratios.csv"
filename_volumes = "./data/putcall_volumes.csv"
date_format = "%Y-%m-%d"
today = datetime.datetime.today().strftime(date_format)

In [2]:
tables = pd.read_html(url)

In [3]:
ratios_df = tables[0]
ratios_df.reset_index(drop=False, inplace=True)
ratios_df = ratios_df.T
ratios_df.columns = ratios_df.iloc[1]
ratios_df = ratios_df.drop(ratios_df.index[[0, 1]])
ratios_df['DATE'] = today

In [4]:
ratios_df.head()

RATIOS,TOTAL PUT/CALL RATIO,INDEX PUT/CALL RATIO,EXCHANGE TRADED PRODUCTS PUT/CALL RATIO,EQUITY PUT/CALL RATIO,CBOE VOLATILITY INDEX (VIX) PUT/CALL RATIO,SPX + SPXW PUT/CALL RATIO,OEX PUT/CALL RATIO,MRUT PUT/CALL RATIO,DATE
RATIOS.1,0.77,1.47,0.83,0.44,1.26,1.5,0.85,20.26,2021-04-17


In [5]:
def check_date_in_file(filename, today):
    try:
        file_df = pd.read_csv(filename, index_col=['DATE'], parse_dates=['DATE'])
        if (file_df.index == today).any():
            print ('Date exist in the file') 
            return True
        else:
            return False
    except OSError as e:
        # File doesn't exist, create a new one
        return False

if not check_date_in_file(filename_ratios, today):
    with open(filename_ratios, 'a') as f:
        ratios_df.to_csv(f, header=f.tell()==0, index=False)

Date exist in the file


In [6]:
volume1_df = tables[1]
volume1_df['TYPE'] = 'SUM OF ALL PRODUCTS'
volume1_df.reset_index(drop=True, inplace=True)
volume1_df['DATE'] = today
volume1_df.columns = ['MEASURE','CALL','PUT','TOTAL','TYPE','DATE']
volume1_df = volume1_df.drop(volume1_df.index[[0]])
volume1_df

,MEASURE,CALL,PUT,TOTAL,TYPE,DATE
1,VOLUME,3743712,2866828,6610540,SUM OF ALL PRODUCTS,2021-04-17
2,OPEN INTEREST,243604627,187098520,430703147,SUM OF ALL PRODUCTS,2021-04-17


In [7]:
volume_columns = ['MEASURE','CALL','PUT','TOTAL','TYPE','DATE']
volume_df = pd.DataFrame(columns=volume_columns)
type_data =['SUM OF ALL PRODUCTS', 'INDEX OPTIONS', 'EXCHANGE TRADED PRODUCTS', 'EQUITY OPTIONS', 'CBOE VOLATILITY INDEX (VIX)', 'SPX + SPXW', 'OEX', 'MRUT']
def get_volume_tables(tables, type_data, df):
    for table_num in range(1, len(type_data)):
        print("Processing: "+str(table_num)+" Name: "+type_data[table_num])
        volume_table_df = tables[table_num]
        volume_table_df['TYPE'] = type_data[table_num]
        volume_table_df.reset_index(drop=True, inplace=True)
        volume_table_df['DATE'] = today
        volume_table_df.columns = volume_columns
        volume_table_df = volume_table_df.drop(volume_table_df.index[[0]])
        df = pd.concat([df, volume_table_df])
    return df
volume_df = get_volume_tables(tables, type_data, volume_df)

Processing: 1 Name: INDEX OPTIONS
Processing: 2 Name: EXCHANGE TRADED PRODUCTS
Processing: 3 Name: EQUITY OPTIONS
Processing: 4 Name: CBOE VOLATILITY INDEX (VIX)
Processing: 5 Name: SPX + SPXW
Processing: 6 Name: OEX
Processing: 7 Name: MRUT


In [8]:
if not check_date_in_file(filename_volumes, today):
    with open(filename_volumes, 'a') as f:
        volume_df.to_csv(f, header=f.tell()==0, index=False)

Date exist in the file
